In [ ]:
import json
import os
import pickle
import re
import sys
sys.path.append('../')
from glob import glob

import editdistance
import numpy as np
import pandas as pd
from pyannote.audio import Model
from pyannote.audio.pipelines import VoiceActivityDetection
from pyannote.core import notebook, Segment
from pydub import AudioSegment
from tqdm import tqdm
from whisper_model import WhisperASR
import matplotlib.pyplot as plt

import scipy.io.wavfile as wav
import scipy.signal as signal

import tempfile




import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

In [ ]:
# trim the audio using start end end time in secs
def trim_audio(path, start, end, out_path):
    sound = AudioSegment.from_file(path, format="wav")
    # make sure that the start and end are in between the audio duration
    start = max(0, start)
    end = min(end, len(sound) / 1000)
    trimmed_sound = sound[start * 1000 : end * 1000]
    trimmed_sound.export(out_path, format="wav")
    return out_path, start, end



In [ ]:
# files_folder = "/data/tts-qa/tts-data/French(Dorsaf)/trimmed"
files_folder = "/data/tts-qa/tts-data/German(Dorothee)/trimmed"
# files_folder = "/data/tts-qa/tts-data/English(Melynda)/trimmed"
# files_folder = "/data/tts-qa/tts-data/Italian(Martina) Deliverable 3/trimmed"
# files_folder = "/data/tts-qa/tts-data/Spanish(Violeta) Deliverable 3/trimmed"


files = glob(os.path.join(files_folder, "*.wav"))


In [ ]:
# randomly select  files

np.random.seed(10)
selected_files = np.random.choice(files, 50, replace=False)

# selected_files = [
#     "/data/tts-qa/tts-data/French(Dorsaf)/trimmed/FR00000026.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/trimmed/FR00000032.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/trimmed/FR00000033.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/trimmed/FR00000035.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/trimmed/FR00001001.wav",
# ]


# rare_cases = [
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000907.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000115.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000169.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000952.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000584.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000911.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00001238.wav",
#     "/data/tts-qa/tts-data/French(Dorsaf)/raw/FR00000979.wav",
# ]
# selected_files= rare_cases

# selected_files = [
#     "/data/tts-qa/tts-data/German(Dorothee)/trimmed/DE00080623.wav",
#     "/data/tts-qa/tts-data/German(Dorothee)/trimmed/DE00069958.wav",
# ]


In [ ]:
from src.utils.audio import CustomVAD
my_custom_vad = CustomVAD(pyannote_model_path="pyannote/segmentation", silero_model_path="snakers4/silero-vad")


In [ ]:
# run for selected files and plot
for file in selected_files[:10]:
    response = my_custom_vad.process_file(file)

    # load the waveform
    waveform = response["resampled_waveform"]
    # Calculate time vector
    time_vector = np.linspace(0, len(waveform) / my_custom_vad.SAMPLING_RATE, num=len(waveform))

    # plot 
    fig = plt.figure(figsize=(20, 5), dpi=50)
    plt.yticks([])
    plt.plot(time_vector, waveform)

    # add the filename as title
    plt.title(os.path.basename(file))

    # plot pyannote
    plt.axvspan(response["pyannote_segment"][0], response["pyannote_segment"][1], color="green", alpha=0.3, label="PYANNOTE")

    # plot silero
    plt.axvspan(response["silero_segment"][0], response["silero_segment"][1], color="red", alpha=0.3, label="SILERO-VAD")

    # plot custom
    plt.axvspan(response["custom_segment"][0], response["custom_segment"][1], color="black", alpha=0.3, label="MY_CUSTOM_VAD")

    plt.legend()
    plt.show()
    


In [ ]:
response

In [ ]:
trim_start, trim_end = tuple(response['custom_segment'])

In [ ]:
trim_start

In [ ]:
trim_end